In [1]:
"""
In this script, we will see how to define a custom model and train it using the Mammoth library.

In addition to the usual `train` and `load_runner` functions, we will need:
- `register_model`: to register our custom model with the Mammoth library.
- `ContinualModel`: to define our custom model.
"""

from mammoth_lite import register_model, ContinualModel, load_runner, train

In [2]:
@register_model('new-sgd') # Register this model with the name 'new-sgd'
class NewSgd(ContinualModel):
    """
    Each model must inherit from ContinualModel and implement the observe method.

    The observe method is called every time a new batch of data is available.
    It is responsible for training the model on the current task using the data provided.

    The model can also include a `COMPATIBILITY` attribute to specify the scenarios it is compatible with.
    In Mammoth-Lite, only the 'class-il' and 'task-il' scenarios are available and are set as default.

    The ContinualModel class provides attrubutes such as:
    - `net`: the backbone model that is used for training. The backbone is defined by default in the dataset but can be changed with the `backbone` argument.
    - `opt`: the optimizer used for training.
    - `loss`: the loss function used for training. This is defined by the dataset and is usually CrossEntropyLoss.
    """
    # COMPATIBILITY = ['class-il', 'task-il'] # More scenarios are available in the full Mammoth repo.

    def observe(self, inputs, labels, not_aug_inputs, epoch=None):
        """
        We will implement just the simplest algorithm for Continual Learning: SGD.
        With SGD, we simply train the model with the data provided, with no countermeasures against forgetting.
        """
        # zero the gradients
        self.opt.zero_grad()

        # forward pass on the model
        outputs = self.net(inputs)

        # compute the loss
        loss = self.loss(outputs, labels)

        # backward pass
        loss.backward()

        # update the weights
        self.opt.step()

        # return the loss value, for logging purposes
        return loss.item()


In [3]:
"""
Now we can use the `load_runner` function to load our custom model.
"""

model, dataset = load_runner('new-sgd','seq-cifar10',{'lr': 0.1, 'n_epochs': 1, 'batch_size': 32})
train(model, dataset)

Loading model:  new-sgd
- Using ResNet as backbone
Using device cuda


  0%|          | 0/313 [00:00<?, ?it/s]

Task 1


Evaluating Task 1: 100%|██████████| 63/63 [00:00<00:00, 89.18it/s, acc_task_1=68.2] 


Accuracy for task 1	[Class-IL]: 68.20 	[Task-IL]: 68.20


...

Accuracy for task 4	[Class-IL]: 18.20 	[Task-IL]: 63.24


  0%|          | 0/313 [00:00<?, ?it/s]

Task 5


Evaluating Task 5: 100%|██████████| 315/315 [00:03<00:00, 87.87it/s, acc_task_5=82.5] 

Accuracy for task 5	[Class-IL]: 16.50 	[Task-IL]: 68.13
